In [1]:
import torch
print(torch.cuda.is_available()) 
from Supporting_functions import *
import time

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'        
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

True


# Predictions for a folder

In [2]:
# Define paths

modelName = 'Nuclei'
epoch = 98
itter = 10000
modelId = f'deblur_net_{epoch}_{itter}.pkl'

# Models
srcpath = r'\\WS2\WS2_Remote_Disk\Current Segovia Lab\CRISTIAN_TEMP\models\Control\Control\Self_'+modelName+'/'
model_path = srcpath+'checkpoint/saved_models/'+modelId
# print(os.listdir(srcpath+'checkpoint/saved_models/'))
# Image to test
img_src_path = r'\\WS1\WS1_Remote_Disk\Current Segovia Lab\Cristian_Perez\Images_to_segment\20221014_8_control_16w\control-16w\02_normalization/'
scale=1

#output dir
outdir = rf'\\WS1\WS1_Remote_Disk\Current Segovia Lab\Cristian_Perez\Images_to_segment\20221014_8_control_16w\control-16w\03_deconvolution/'

# CUDA device
device1 = torch.device('cuda:0')
device2 = torch.device('cuda:0')
batch_size = 16

# Image Normalization
min_v = 0
max_v = 65535
norm_percentiles = (30, 99.999)  #99.9995 For Nuclei, 99.999 For Membranes
thres_scale = 1.5 # threhold for simple mask for normalization

# Create output folder
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [3]:
# Prepare networks
deblur_net_A = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')
deblur_net_B = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1,use_dropout=False,norm='instance')

#deblur_net_A  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1, use_dropout=False, norm='instance')
#deblur_net_B  = Self_net_architecture.define_G(input_nc=1, output_nc=1, ngf=64, netG='deblur_net', device=device1, use_dropout=False, norm='instance')


 # Load Model
deblur_net_A.load_state_dict(torch.load(model_path))
deblur_net_B.load_state_dict(torch.load(model_path))
    

# Get all tif images in the folder
image_names = [f for f in os.listdir(img_src_path) if f.endswith('.tif')]

# Calculate for each model
for i, image_name in enumerate(image_names):

    start_time = time.time()  # Record the start time   
    
    # Open image
    img_path = os.path.join(img_src_path, image_name)
    img  =  65535 * tifffile.imread(img_path).astype(np.float32)

    # Normalize 
    img = image_normalization(img, norm_percentiles, thres_scale, min_v, max_v)
    img = img.astype(np.float32)
    
    # Predict image
    fusion_stack = upsample_block(img,scale,1,deblur_net_A,deblur_net_B,min_v,max_v, device1, device2,batch_size)

    
    # Image Normalization
    fusion_stack = image_normalization(fusion_stack, norm_percentiles, thres_scale, min_v, max_v)
    
    # Save image
    img_out = os.path.join(outdir, image_name)
    tifffile.imwrite(img_out,fusion_stack)
    
    elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {elapsed_time:.4f} seconds, image {image_name}")   
    


initialize network with normal
initialize network with normal


C:\Users\WORKSTATION-PC\AppData\Local\Temp\ipykernel_2412\1483027444.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  deblur_net_A.load_state_dict(torch.load(model_path)

mode_result.mode :  65535.0
     -threshold_value: 98302.5
Intensity Norm  from (0 , 4272161024) to  (0, 65535)  



Predicting : 100%|██████████████████████████████████████████████████████████████| 106/106 [00:38<00:00,  2.73 planes/s]


mode_result.mode :  0
     -threshold_value: 0.0
Intensity Norm  from (0 , 60804) to  (0, 65535)  

Elapsed Time: 179.5563 seconds, image DAPI_corrected.tif


In [4]:
print(img_out)

\\WS1\WS1_Remote_Disk\Current Segovia Lab\Cristian_Perez\Images_to_segment\20221014_8_control_16w\control-16w\03_deconvolution/DAPI_corrected.tif


In [5]:
#tifffile.imwrite(img_out,fusion_stack)